In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import os 
import sys

from astropy.coordinates import SkyCoord
from datetime import datetime


In [ ]:
sys.path.insert(0, '/arc/home/mseth/beam-model')
from beam_model import composite as cmp

sys.path.insert(0, '/arc/home/mseth/frb-calibration-master')
import frb_calibration.calibration_scheduler_helpers as cal_scheduler

 Load in data

In [ ]:
data = np.load('/arc/projects/chime_frb/mseth/cyg_A/frb_CYG_A_2025-07-03_beam_1105.npz', allow_pickle=True) 

In [ ]:
spectra = data['spectra']
median_timeseries = data['median_timeseries']
ts = spectra.mean(spectra, axis=0)

In [ ]:
# Find index of where average timeseries peaks  
peak_idx = np.argmax(np.mean(ts))

# Get the spectra for that time
spectra_at_peak = spectra[:,peak_idx]


Get sensitivities

In [ ]:
source_name = "CYG_A"
coords = SkyCoord.from_name(source_name)

source_ra = coords.ra.deg
source_dec = coords.dec.deg

In [ ]:
# Finding which beams are transited and what times 

time = datetime(2025, 7, 3)

transited_beam_ids = cal_scheduler.calculate_transit_beams_precise(
    source_ra, source_dec)

transit_times = cal_scheduler.calculate_next_transit_times_precise(
    source_ra, source_dec, time, transited_beam_ids)

print("Precise beams transited: {}".format(sorted(transited_beam_ids)))
print("Precise transit times: {}".format(transit_times))

In [ ]:
# Pick out the time it transits our beam 
transit_time = transit_times[2]

In [ ]:
# Make an array of times and positions
start_time = transit_time - pd.Timedelta(minutes=10)
end_time = transit_time - pd.Timedelta(minutes=10)

datetime_arr = pd.date_range(start=start_datetime, end=end_datetime)

pos_list = []
for i in datetime_arr:
    pos = utils.get_position_from_equatorial(source_ra, source_dec, i)
    pos_list.append(pos)

pos_array = np.array(pos_list)

In [ ]:
#Calculate sensitivity 
cbm = composite.CompositeBeamModel(config.current_config)
freqs = np.arange(400, 800, 16384)

sensitivity = cbm.get_sensitivity(1105, pos_array, freqs)

#Find index of where mean sensitivity is maximum 
max_sens_idx = np.argmax(np.mean(sensitivity))

#Get sensitivity spectrum at max index 
sensitivity_at_peak = sensitivity[max_sens_idx,:]


Correcting

In [ ]:
corrected_spectra = spectra_at_peak / sensitivity_at_peak

In [ ]:
# Define conversion equation

def bf_to_jy(bf_int, f_good):
    factor = (np.square(1024) * 128) / (np.square(4) * 0.806745 * 400)
    result = bf_int / ( factor * np.square(f_good) ) 
    return result

jansky_spectra = bf_to_jy(corrected_spectra, 0.9)

In [ ]:
# Make a function for the spectrum (from Perley & Butler 2016)

def spectrum(a_0, a_1, a_2, a_3, GHz):
    exponent = a_0 + (a_1 * np.log10(GHz)) + (a_2 * np.square(np.log10(GHz))) + (a_3 * np.power(np.log10(GHz),3))
    result = 10**exponent)
    return result 

cygA_spectrum = spectrum(a_0 = 3.34598,
                         a_1 = -1.0022,
                         a_2 = -0.225,
                         a_3 = 0.023,
                         #a_4= 0.043,
                         GHz = freqs/1000)

In [ ]:
# Plot corrected spectrum vs. modeled spectrum 
plt.figure()
plt.plot(freqs/1000, jansky_spectra, label="Corrected Spectrum at Peak")
plt.plot(freqs/1000, cygA_spectrum, label='Model Spectrum')
plt.yscale('log')
plt.xscale('log') 
plt.xlabel('Frequency(GHz)')
plt.ylabel('Flux density(Jy)')
plt.show()